# 0. NOTES

* feature `real_face` could be removed, it provides no extra information to the dataset
* without `player_name` information we loose track of how many unique instances there could be

# 1. Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# 2. Data importing

In [3]:
data = pd.read_csv('../../raw_data/master_df_with_webscraping.csv')

In [4]:
data.head()

,fee_cleaned,age,height_cm,weight_kg,nationality,club_name,league_name,league_rank,overall,potential,...,lb,lcb,cb,rcb,rb,fifa year,national_team,seasons_with_club,on_loan,remaining_seasons_on_contract
0,0.069,17,178,57,Wales,Newport County,English League Two,4.0,41,50,...,32,31,31,31,32,2015,0,1,0,0.0
1,2.700,27,183,73,Germany,VfL Wolfsburg,German 1. Bundesliga,1.0,78,78,...,59,53,53,53,59,2015,0,1,0,2.0
2,5.400,27,165,63,England,Tottenham Hotspur,English Premier League,1.0,79,79,...,50,41,41,41,50,2015,0,10,0,1.0
3,1.530,24,178,71,Guinea,Angers SCO,French Ligue 2,2.0,69,73,...,49,45,45,45,49,2015,0,1,0,2.0
4,9.540,21,182,69,France,Stade Rennais FC,French Ligue 1,1.0,70,78,...,65,66,66,66,65,2015,0,4,0,2.0
